In [1]:

import os
from dotenv import load_dotenv
from langchain_ollama import OllamaEmbeddings

# 1.加载环境变量
load_dotenv()

# 2.加载模型
embedding_model = OllamaEmbeddings(
    base_url=os.environ['OLLAMA_BASE_URL'],  # 可配置为内部服务器地址
    model=os.environ['OLLAMA_EMBEDDING_MODEL_NAME'],
    temperature=0.8,
    num_ctx=4096  # 上下文窗口大小
)


In [ ]:
import os
import jieba.analyse
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader, PDFPlumberLoader
from langchain_chroma import Chroma

knowledge_base = {
    # "policy": "../documents/policy",
    "quality": "../documents/quality",
    "history": "../documents/history"
}

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
    separators=["\n\n", "\n", ". ", "! ", "? ", ", "]
)

def _enhance_documents(docs, subject):
        """文档增强处理（添加元数据和学科标签）"""
        enhanced = []
        for doc in docs:
            # 添加学科元数据
            doc.metadata.update({
                "subject": subject,
                "keywords": " ".join(jieba.analyse.extract_tags(doc.page_content, topK=5))
            })
            # 在内容前添加学科标识
            doc.page_content = f"[{subject}] {doc.page_content}"
            enhanced.append(doc)
        print(f"已增强 {len(enhanced)} 个文档块", enhanced[:3])
        return enhanced

def build_and_save():
        """构建并持久化向量数据库"""
        all_docs = []
        
        for subject, dir_path in knowledge_base.items():
            if not os.path.exists(dir_path):
                print(f"跳过不存在的目录：{dir_path}")
                continue
            
            # 加载和分割文档
            loader = DirectoryLoader(
                dir_path,
                glob="**/*.pdf",
                loader_cls=PDFPlumberLoader,
                show_progress=True
            )
            raw_docs = loader.load()
            split_docs = text_splitter.split_documents(raw_docs)
            
            # 执行文档增强
            enhanced_docs = _enhance_documents(split_docs, subject)
            all_docs.extend(enhanced_docs)
            print(f"已处理 {len(enhanced_docs)} 个 {subject} 文档块")
        return all_docs

all_docs = build_and_save()


100%|██████████| 1/1 [02:20<00:00, 140.90s/it]


已增强 427 个文档块 [Document(metadata={'source': '../documents/quality/2025quality.pdf', 'file_path': '../documents/quality/2025quality.pdf', 'page': 0, 'total_pages': 427, 'Author': '', 'Comments': '', 'Company': '', 'CreationDate': "D:20250121112430+08'00'", 'Creator': 'WPS 演示', 'Keywords': '', 'ModDate': "D:20250121112430+08'00'", 'Producer': '', 'SourceModified': "D:20250121112430+08'00'", 'Subject': '', 'Title': '', 'Trapped': 'False', 'subject': 'quality', 'keywords': '质量 管理'}, page_content='[quality] 质量管理'), Document(metadata={'source': '../documents/quality/2025quality.pdf', 'file_path': '../documents/quality/2025quality.pdf', 'page': 1, 'total_pages': 427, 'Author': '', 'Comments': '', 'Company': '', 'CreationDate': "D:20250121112430+08'00'", 'Creator': 'WPS 演示', 'Keywords': '', 'ModDate': "D:20250121112430+08'00'", 'Producer': '', 'SourceModified': "D:20250121112430+08'00'", 'Subject': '', 'Title': '', 'Trapped': 'False', 'subject': 'quality', 'keywords': '管理 质量 框架 TQM 资源管理'}, page

100%|██████████| 1/1 [00:23<00:00, 23.61s/it]


已增强 815 个文档块 [Document(metadata={'source': '../documents/history/15040.pdf', 'file_path': '../documents/history/15040.pdf', 'page': 0, 'total_pages': 815, 'Title': 'PowerPoint 演示文稿', 'Author': 'HouYujia', 'CreationDate': "D:20231030143608+08'00'", 'ModDate': "D:20241224105232+08'00'", 'Producer': 'Microsoft® PowerPoint® 2016', 'Creator': 'Microsoft® PowerPoint® 2016', 'subject': 'history', 'keywords': '习近平 概论 特色 社会主义 思想'}, page_content='[history] 习近平新时代中国特色社会主义思想概论'), Document(metadata={'source': '../documents/history/15040.pdf', 'file_path': '../documents/history/15040.pdf', 'page': 1, 'total_pages': 815, 'Title': 'PowerPoint 演示文稿', 'Author': 'HouYujia', 'CreationDate': "D:20231030143608+08'00'", 'ModDate': "D:20241224105232+08'00'", 'Producer': 'Microsoft® PowerPoint® 2016', 'Creator': 'Microsoft® PowerPoint® 2016', 'subject': 'history', 'keywords': '全面 社会主义 坚持 现代化 特色'}, page_content='[history] 习近平新时代中国特色社会主义思想概论\n课程目录\n导论\n第一章 新时代坚持和发展中国特色社会主义\n第二章 以中国式现代化全面推进中华民族伟大复兴\n第三章 坚持党的全面领导\

TypeError: Chroma.__init__() got an unexpected keyword argument 'metadata'

In [ ]:
# 3.构建向量数据库
def build_vector_store(all_docs, persist_dir="../vector_store/knowledge_base_one"):
    vector_store = Chroma.from_documents(
        all_docs,
        embedding_model,
        persist_directory=persist_dir,
        collection_metadata={"hnsw:space": "cosine"},
    )
    return vector_store

build_vector_store(all_docs)

In [4]:
# 获取向量数据库
from langchain_chroma import Chroma
vector_store = Chroma(persist_directory="../vector_store/knowledge_base_one", embedding_function=embedding_model)

In [18]:
# retriever_result = vector_store.similarity_search_with_score("质量管理原则")
# for doc, score in retriever_result:
#     print(score, doc)
# 输出分隔符
print("*" * 50)
# retriever_result2 = vector_store.similarity_search_with_score("全面依法治国")

# for doc, score in retriever_result2:
#     print(score, doc)
# print(retriever_result2)

**************************************************


In [26]:
# 检索器构造
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3, "filter": None}
)
def query(question, category=None):
        search_kwargs = {"k": 3}
        if category:
            search_kwargs["filter"] = {"subject": category}
        print(search_kwargs)
        return retriever.invoke(question,search_kwargs)
# 混合检索
mix_retriever = vector_store.as_retriever(
    search_type="mmr",  # 最大边际相关性
    search_kwargs={"k": 5, "lambda_mult": 0.7}
)
# results = query("全面依法治国", "quality")
results = mix_retriever.invoke("全过程民主")
result = "\n".join([doc.page_content for doc in results])
print(result)

[history] 第八章 发展全过程人民民主
[history] 第二节 全过程人民民主是社会主
义民主政治的本质属性
[history] 第二节 全过程人民民主是社会主义民主政治的本质属性
知识点3 全过程人民民主是全链条、全方位、全覆盖的民主
（1）全过程人民民主是全链条的民主。在民主选举环节，人民通过选
举投票行使权利，选出代表自己意愿的人来掌握并行使权力。
（2）全过程人民民主是全方位的民主。全过程人民民主贯通国家政治
生活和社会生活各层面各维度。
（3）全过程人民民主是全覆盖的民主。全过程人民民主涵盖国家各项
事业各项工作，贯穿“五位一体”总体布局和“四个全面”战略布局
的方方面面，是各方面民主的有机统一。
[history] 第八章 发展全过程人民民主
本章重难点分析
1.人民民主是社会主义的生命
2.中国特色社会主义政治制度行得通、有生命力、有效率
3.坚定不移走中国特色社会主义政治发展道路
4.全过程人民民主是社会主义民主政治的伟大创造
5.全过程人民民主是全链条、全方位、全覆盖的民主
6.全过程人民民主是最广泛、最真实、最管用的民主
[history] 第三节 健全人民当家作主的制度体系
知识点2 全面发展协商民主
社会主义协商民主，是在中国共产党领导下，人民内部各方面围
绕改革发展稳定重大问题和涉及群众切身利益的实际问题，在决策之
前和决策实施之中开展广泛协商，努力形成共识的重要民主形式。
协商民主源自中华民族长期形成的天下为公、兼容并蓄、求同存
异等优秀政治文化，源自近代以后中国政治发展的现实进程，源自中
国共产党领导人民进行革命、建设、改革的长期实践，源自新中国成
立后各党派、各团体各民族、各阶层、各界人士在政治制度上共同实
现的伟大创造，源自改革开放以来中国在政治体制上的不断创新，具
有深厚的文化基础、理论基础、实践基础、制度基础。


'检索结果: 如何使用langchain? 请给出详细的技术解决方案'